In [56]:
import pandas as pd
from scipy import sparse
import pandas as pd
import numpy as np
from numpy import var
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy import stats
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statistics
import math
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
import random
import copy
from gensim.parsing.preprocessing import remove_stopwords
from scipy.stats import kurtosis
from scipy.stats import skew


#init
random.seed(1)

#load data
ratings = pd.read_csv('dataset/ratings.csv')
moviesPartial = pd.read_csv('dataset/movies.csv')
moviesFull = pd.read_csv('dataset/movie_dataset.csv')

#drop duplicates
moviesFull.drop_duplicates(subset = 'title', inplace=True)

#drop columns that do nothing
ratings = ratings.drop(['timestamp'], axis = 1)
moviesPartial = moviesPartial.drop(['genres'],axis=1)

#all the users are to be tested
userRatings = []
for i in range(1,611):
    userRatings.append(ratings[ratings['userId'] ==i])

#remove the year and keep the name
moviesPartial['title'] = moviesPartial['title'].map(lambda string: string[:-7])

#drop duplicates
moviesPartial.drop_duplicates(subset = 'title', inplace=True)


#helper functions:
def get_title_from_movieId(tmp):
    return moviesPartial[moviesPartial["movieId"]== tmp]["title"].to_string(index=False)

def checkTitle(tmp):
    if(tmp not in list(moviesFull["title"])):
        #The word "the" is being apended to the ends of titles
        tmp = tmp[-3:] +" "+ tmp[:-5]
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)
    else:
        return moviesFull[moviesFull["title"] == tmp]["title"].to_string(index=False)



#used to filter out movies that are not in both data sets
newUserTitles = []
newUserRatings = []
averageUserRating = 0
cnt =0
for userRating in userRatings:
    temp1 = []
    temp2 = []
    for rating, id in zip(list(userRating["rating"]), list(userRating["movieId"])):
        title = get_title_from_movieId(id)
        checked = checkTitle(title)
        if(checked != 'Series([], )'):
            temp1.append(checked)
            temp2.append(rating)
            averageUserRating += rating
            cnt+=1

    newUserTitles.append(temp1)
    newUserRatings.append(temp2)

averageUserRating/= cnt
    


infos = []
for user in newUserTitles:
    i = 0
    tempInfo = pd.DataFrame(columns = list(moviesFull.columns))
    for name in user:
        if(len(moviesFull[moviesFull["title"] == name]) !=0):
            tempInfo.loc[i] = moviesFull[moviesFull["title"] == name].iloc[0]
        i+=1
    infos.append(tempInfo)
    

features = ['keywords','cast','genres','director']


for info in infos:
    for feature in features:
        info[feature] = info[feature].fillna('')

colNumbers = []
for feature in features:   
    colNumbers.append(infos[0].columns.get_loc(feature))

fullDataList =[]


for frame in infos:
    fullDataList.append(frame.values.tolist())

combinedFeatures = []

for item in fullDataList:
    tmp = []
    for row in item:
        cleaned = remove_stopwords(row[colNumbers[0]]+" "+row[colNumbers[1]]+" "+row[colNumbers[2]]+" "+row[colNumbers[3]])
        tmp.append(cleaned)
    combinedFeatures.append(tmp)


def transform1(arr):
    ret = []
    for item in arr:
        if(item<=.5):
            ret.append(1/item)
        if(item>.5):
            ret.append(-1*(1/(1-item)))
    return ret

def transform2(arr):
    ret = []
    for item in arr:
        ret.append(1/item)
    return ret


def findAverageRating(arr1, arr2, title, ignore1, ignore2, ave):
    ret =0
    cnt =0
    single = True
    for index, (item1, item2) in enumerate(zip(arr1, arr2)):
        if(len(item1)!=len(item2)):
            print("diff", len(item1), len(item2))
        for j in range(len(item1)):
            if(title == item1[j] and (ignore1!=index or ignore2!=j)):
                ret+= item2[j]
                cnt+=1
                single = False

    if(single):
        return ave
    else:
        return ret/cnt



X = []
y = []

for i in range(len(combinedFeatures)):
    count_matrix = CountVectorizer().fit_transform(combinedFeatures[i]).toarray().tolist()
    randIndex = random.randint(0, len(count_matrix)-1)
    randTestItem = count_matrix[randIndex]
    del count_matrix[randIndex]

    cosine_sim = cosine_similarity(X = count_matrix ,Y = [randTestItem])

    ratings = copy.deepcopy(newUserRatings[i])
    randomRating = ratings[randIndex]
    del ratings[randIndex]


    similairities = np.reshape(cosine_sim,  (len(cosine_sim)))
    averageRating =  sum(ratings)/(len(ratings))

    tmp = []

    param = findAverageRating(newUserTitles, newUserRatings, newUserTitles[i][randIndex],i, randIndex, averageUserRating)
    tmp.append(param)
    tmp.append(averageRating)
    tmp.append(sum(similairities)/(len(similairities)))
    tmp.append(statistics.variance(ratings))
    tmp.append(statistics.variance(similairities))

    skew1= skew(ratings)
    skew2 = skew(similairities)
    kurt1 = kurtosis(ratings)
    kurt2 = kurtosis(similairities)


    lst = []
    for item in [skew1,skew2, kurt1, kurt2]:
        if(math.isnan(item)):
            lst.append(0)
        else:   
            lst.append(item)

    print("skew1",lst[0])
    print("skew2",lst[1])
    print("kurt1",lst[2])
    print("kurt2",lst[3])

    tmp.append(lst[0])
    tmp.append(lst[1])
    tmp.append(lst[2])
    tmp.append(lst[3])


    # ratingsZ = stats.zscore(ratings)
    # ratingsZ = [0 if math.isnan(x) else x for x in ratingsZ]
    # ratingsZ = [-item if item<0 else item for item in ratingsZ]
    # p_values = norm.sf(ratingsZ)
    # transformed2 = transform2(p_values)
    # sum2 = sum(transformed2)
    # simsZ  = stats.zscore(similairities)
    # simsZ = [-item if item<0 else item for item in simsZ]

    totalRating = 0


    for sim, rating in zip(similairities, ratings):
        totalRating += sim*(rating-averageRating)
    

    tmp.append(totalRating)
    X.append(tmp)
    y.append(randomRating)




skew1 -1.2523530403187937
skew2 2.1860784557753727
kurt1 1.6646772178259024
kurt2 5.292011875138016
skew1 -0.3261256343442037
skew2 1.206070193874346
kurt1 -0.7712066115702463
kurt2 1.1290387966381745
skew1 1.077627629554531
skew2 0.7505360090710362
kurt1 -0.6151299097403871
kurt2 -0.7435771276813647
skew1 -0.5552538606437774
skew2 1.4629151552733843
kurt1 -0.9437543893744063
kurt2 2.176645950474456
skew1 -0.28849468542975987
skew2 2.8923927394388382
kurt1 -0.13613928038126133
kurt2 10.945789379770538
skew1 -0.37339372025727424
skew2 0.7575968090157691
kurt1 0.08339454530237056
kurt2 1.1275047772531375
skew1 -0.4440412038223139
skew2 1.1527884782379538
kurt1 -1.1102436010244443
kurt2 1.8497139222060301
skew1 -0.01951099966363523
skew2 0.031437744716198236
kurt1 -1.1494311637080867
kurt2 -0.6060582808755823
skew1 -0.46083744918325675
skew2 0.896659545867851
kurt1 -0.7711925476399641
kurt2 0.24605748936922156
skew1 -0.8282713833498425
skew2 1.3590353785922047
kurt1 0.32150407536594816
ku

C:\Users\jackson\AppData\Local\Temp\ipykernel_5760\3196615624.py:194: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skew1= skew(ratings)
C:\Users\jackson\AppData\Local\Temp\ipykernel_5760\3196615624.py:196: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt1 = kurtosis(ratings)


skew1 -0.2794670895389949
skew2 1.0473120649080432
kurt1 -0.6404879245390154
kurt2 1.0232201877094296
skew1 -0.4722152350199524
skew2 0.6044312424044127
kurt1 -0.23010490758213642
kurt2 0.16459452668964492
skew1 -1.1836800470409845
skew2 1.1298265450100895
kurt1 0.5596852129855532
kurt2 2.728278190133687
skew1 0.5383323231255601
skew2 0.9093171257003849
kurt1 -1.044581618655694
kurt2 0.3341786599219838
skew1 -1.9456023692099698
skew2 3.470930652432798
kurt1 4.056220046466566
kurt2 10.899034408046752
skew1 -1.3684937146863116
skew2 2.194628475194461
kurt1 3.3356871547869433
kurt2 6.650022888952318
skew1 -0.7017519484394898
skew2 0.5562025742159404
kurt1 0.5020987146420559
kurt2 0.1671946679964731
skew1 -0.8323691547783335
skew2 0.9402285321363687
kurt1 2.271593613488787
kurt2 0.7667191877680937
skew1 -0.7796959099782721
skew2 1.9899770242132941
kurt1 -0.5249891062644596
kurt2 5.82560344729135
skew1 -0.8415128537866957
skew2 6.438235250433454
kurt1 2.6706948558871497
kurt2 63.17218460781

In [108]:
#https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
#predicting imbd ratings website:
#https://jazpeng.github.io/predict_movie_ratings/

#need to cross validate!!!
#need to use krutosis and skewness:
#https://www.google.com/search?q=what+is+kurtosis&rlz=1C1ONGR_enUS1027US1027&oq=what+is+kurtosis&aqs=chrome..69i57j0i512j0i20i263i512j0i512l7.3563j0j7&sourceid=chrome&ie=UTF-8
#https://www.statisticshowto.com/probability-and-statistics/descriptive-statistics/skewness/#:~:text=The%20formula%20given%20in%20most,%E2%80%93%20Median)%20%2F%20Standard%20Deviation.
#https://datascience.stackexchange.com/questions/36291/using-neural-network-regression-for-skewed-data


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=1)
print(X)
print(y)

regr = MLPRegressor(hidden_layer_sizes=(500, 500), activation = "tanh", solver ="sgd",  max_iter=1000, random_state =1).fit(X_train, y_train)

score  = regr.score(X_test, y_test)
file = open("testing.txt", "a")
file.write("hidden_layer_sizes="+str(regr.hidden_layer_sizes)+", max_iter = "+str(regr.max_iter)+" " + str(score)+"\n")
file.close()

print("after nearal net regression:", score)
print(regr.max_iter)
print(regr.n_iter_)
print(regr.hidden_layer_sizes)

predicted = regr.predict(X_test)
for item, actual in zip(predicted, y_test):
    print("predicted:", item)
    print("actual:", actual)

[[3.2560975609756095, 4.363636363636363, 0.04167949551131953, 0.6274007682458387, 0.0026695513558354467, -1.2523530403187937, 2.1860784557753727, 1.6646772178259024, 5.292011875138016, 0.09124091886536033], [3.95, 4.043478260869565, 0.06770729506301222, 0.5434782608695652, 0.003507670935774326, -0.3261256343442037, 1.206070193874346, -0.7712066115702463, 1.1290387966381745, 6.525825018766432e-05], [2.872093023255814, 1.5294117647058822, 0.07185603833415556, 2.858455882352941, 0.007309959668710622, 1.077627629554531, 0.7505360090710362, -0.6151299097403871, -0.7435771276813647, 1.6381279873308203], [4.2155172413793105, 3.451923076923077, 0.037641597634876005, 1.8617438386855862, 0.0021643585376757294, -0.5552538606437774, 1.4629151552733843, -0.9437543893744063, 2.176645950474456, -0.6873973238231363], [3.8333333333333335, 3.566666666666667, 0.05399456773509661, 1.0126436781609196, 0.0013718774744848968, -0.28849468542975987, 2.8923927394388382, -0.13613928038126133, 10.945789379770538,

In [63]:
reg = LinearRegression().fit(X_train, y_train)
score  = reg.score(X_test, y_test)

print(score)
print(reg.coef_)
predicted = regr.predict(X_test)
for item, actual in zip(predicted, y_test):
    print("predicted:", item)
    print("actual:", actual)



0.2250006947297738
[ 6.10588813e-01  7.83076301e-01  5.71595934e+00 -7.43952010e-02
 -1.69012730e+01  3.49213782e-02  2.31900316e-01 -1.29615413e-04
 -3.02828330e-02  1.52058787e-01]
predicted: 3.7095079986998845
actual: 2.5
predicted: 4.402664020673592
actual: 3.0
predicted: 4.0574345478301606
actual: 4.5
predicted: 3.5037208505326616
actual: 4.0
predicted: 4.023892346094582
actual: 4.0
predicted: 4.4570577848833866
actual: 5.0
predicted: 3.443367075389045
actual: 2.0
predicted: 3.5605012422917586
actual: 0.5
predicted: 4.420316438014361
actual: 4.5
predicted: 3.1147841123389246
actual: 3.0
predicted: 3.614725395054625
actual: 4.0
predicted: 4.090017476491889
actual: 4.0
predicted: 3.2617966584822464
actual: 3.0
predicted: 3.948909792247582
actual: 3.0
predicted: 2.3229283157100653
actual: 3.0
predicted: 3.7873792447119055
actual: 4.5
predicted: 3.4851185688714144
actual: 3.5
predicted: 3.03547875891574
actual: 3.0
predicted: 3.185246182696678
actual: 3.0
predicted: 4.447754488183105
